In [1]:
import pandas as pd
import mysql.connector
import sqlalchemy

In [293]:
cnx.close()

In [294]:
cnx = mysql.connector.connect(user='root', password='password', 
                              host='127.0.0.1', database='otus_db', port='3306')

In [295]:
cur = cnx.cursor()

#### Создадим таблицу products_info
#### product_id - идентификатор продукта
#### product_name - наименование продукта
#### price - цена продукта
#### properties - свойства продукта
#### description - описание продукта


In [334]:
cur.execute('''
            DROP TABLE IF EXISTS products_info;
            ''')

cur.execute('''
            CREATE TABLE products_info (
                product_id BIGINT UNSIGNED NOT NULL AUTO_INCREMENT PRIMARY KEY,
                product_name VARCHAR(255) NOT NULL,
                price BIGINT UNSIGNED NOT NULL,
                properties VARCHAR(255) NOT NULL,
                description VARCHAR(255) NOT NULL)
                ;
            ''')

In [335]:
cur.execute('''
INSERT INTO products_info (product_name, price, properties, description)
SELECT 
  CASE 
    WHEN RAND() < 0.25 THEN 'testy'
    WHEN RAND() < 0.5 THEN 'very testy food'
    WHEN RAND() < 0.75 THEN 'something cool'
    ELSE 'dont eat it'
  END AS product_name,
  FLOOR(RAND() * 10000) + 1 as price,
  CASE 
    WHEN RAND() < 0.25 THEN 'hot food'
    WHEN RAND() < 0.5 THEN 'drinks'
    WHEN RAND() < 0.75 THEN 'vegetables'
    ELSE 'fruits'
  END AS properties,
  CASE 
    WHEN RAND() < 0.33 THEN 'Made in Russia'
    WHEN RAND() < 0.66 THEN 'Made in USA'
    ELSE 'Made in China'
  END AS description
FROM 
  (SELECT 1 AS n UNION ALL SELECT 2 UNION ALL SELECT 3 UNION ALL SELECT 4) t1
   CROSS JOIN (SELECT 1 AS n UNION ALL SELECT 2 UNION ALL SELECT 3 UNION ALL SELECT 4) t2
   CROSS JOIN (SELECT 1 AS n UNION ALL SELECT 2 UNION ALL SELECT 3 UNION ALL SELECT 4) t3
   CROSS JOIN (SELECT 1 AS n UNION ALL SELECT 2 UNION ALL SELECT 3 UNION ALL SELECT 4) t4
   CROSS JOIN (SELECT 1 AS n UNION ALL SELECT 2 UNION ALL SELECT 3 UNION ALL SELECT 4) t5
   LIMIT 1000;
            ''')

query = f'''
          SELECT
          *
         from products_info
        '''
df = pd.read_sql_query(query, cnx)
df

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,product_id,product_name,price,properties,description
0,1,something cool,6731,vegetables,Made in China
1,2,something cool,315,drinks,Made in USA
2,3,very testy food,9888,vegetables,Made in Russia
3,4,something cool,8493,fruits,Made in USA
4,5,dont eat it,6334,fruits,Made in USA
...,...,...,...,...,...
995,996,something cool,2818,drinks,Made in USA
996,997,very testy food,7592,drinks,Made in China
997,998,testy,5536,hot food,Made in Russia
998,999,testy,9778,vegetables,Made in USA


In [336]:
query = f'''
          SELECT
          distinct
          properties,
          description,
          count(product_id) over (partition by properties, description) as count_products,
          max(price) over (partition by properties, description) as max_price
         from products_info
         where price > 4000
         and properties = 'fruits'
         order by count_products
         limit 3
        '''
df = pd.read_sql_query(query, cnx)
df

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,properties,description,count_products,max_price
0,fruits,Made in China,18,9998
1,fruits,Made in Russia,25,9944
2,fruits,Made in USA,25,9766


#### Создать процедуру выборки товаров с использованием различных фильтров: категория, цена, производитель, различные дополнительные параметры
#### Также в качестве параметров передавать по какому полю сортировать выборку, и параметры постраничной выдачи

In [337]:
cur.execute('''
            drop procedure if exists products_info_count;
            ''')

cur.execute('''
            CREATE procedure  products_info_count(
             IN in_properties VARCHAR(50),
             IN in_price bigint,
             IN in_limit bigint
             
            )
                    BEGIN
                    SELECT
                      distinct
                      properties,
                      description,
                      count(product_id) over (partition by properties, description) as count_products,
                      max(price) over (partition by properties, description) as max_price
                     from products_info
                     where price > in_price
                     and properties = in_properties
                     order by count_products
                     limit in_limit;
                    END;
            ''')

In [338]:
cur.callproc('products_info_count', ['fruits', 4000, 3])
result = []
for result_set in cur.stored_results():
    result.append(result_set.fetchall())
df = pd.DataFrame(result[0], columns=['properties', 'description', 'count_products', 'max_price'])
df

,properties,description,count_products,max_price
0,fruits,Made in China,18,9998
1,fruits,Made in Russia,25,9944
2,fruits,Made in USA,25,9766


#### Создаем пользователя 'client' с паролем 'client_password'
#### Создаем пользователя 'manager' с паролем 'manager_password'

In [213]:
cur.execute('''
            CREATE USER 'client'@'localhost' IDENTIFIED BY 'client_password';
            ''')

cur.execute('''
            CREATE USER 'manager'@'localhost' IDENTIFIED BY 'manager_password';
            ''')

#### Даем права пользователю 'client' на запуск процедуры 'products_info_count'

In [ ]:
cur.execute('''
            GRANT EXECUTE ON PROCEDURE otus_db.products_info_count TO 'client'@'localhost';
            ''')

In [339]:
cur.execute('''
            drop TABLE if exists products_sales 
                ;
            ''')

cur.execute('''
            CREATE TABLE products_sales (
                  id INT AUTO_INCREMENT PRIMARY KEY,
                  product_id BIGINT,
                  date_sales TIMESTAMP,
                  quantity BIGINT
                );
            ''')


cur.execute('''
            INSERT INTO products_sales (product_id, date_sales, quantity)
                SELECT 
                  FLOOR(RAND() * 100) + 1 AS product_id,
                  NOW() - INTERVAL FLOOR(RAND() * 365) DAY AS date_sales,
                  FLOOR(RAND() * 100) + 1 AS quantity
                FROM
                  (SELECT 1 UNION SELECT 2 UNION SELECT 3 UNION SELECT 4 UNION SELECT 5) AS t1,
                  (SELECT 1 UNION SELECT 2 UNION SELECT 3 UNION SELECT 4 UNION SELECT 5) AS t2,
                  (SELECT 1 UNION SELECT 2 UNION SELECT 3 UNION SELECT 4 UNION SELECT 5) AS t3,
                  (SELECT 1 UNION SELECT 2 UNION SELECT 3 UNION SELECT 4 UNION SELECT 5) AS t4
                LIMIT 1000;
            ''')

query = f'''
          SELECT
          *
         from products_sales
        '''
df = pd.read_sql_query(query, cnx)
df

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,product_id,date_sales,quantity
0,1,47,2022-12-11 11:29:11,54
1,2,31,2022-10-24 11:29:11,69
2,3,68,2023-06-04 11:29:11,53
3,4,69,2022-11-20 11:29:11,19
4,5,39,2023-05-17 11:29:11,64
...,...,...,...,...
620,621,89,2023-09-08 11:29:11,56
621,622,65,2023-03-08 11:29:11,82
622,623,45,2022-12-15 11:29:11,55
623,624,38,2023-06-21 11:29:11,18


#### Создать процедуру get_orders - которая позволяет просматривать отчет по продажам за определенный период (час, день, неделя)
#### с различными уровнями группировки (по товару, по категории, по производителю)

In [350]:
query = f'''
          select
          ps.product_id,
          sum(ps.quantity),
          pi.properties,
          pi.description,
          MONTH(date_sales) as time
          from products_sales ps
          left join products_info pi
          on pi.product_id = ps.product_id
          where 
          pi.properties = 'drinks'
          and pi.description = 'Made in Russia'
          and DATE_FORMAT(ps.date_sales, '%Y-%m') = '2023-02'
          group by ps.product_id, time, pi.properties, pi.description
        '''
df = pd.read_sql_query(query, cnx)
df

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,product_id,sum(ps.quantity),properties,description,time
0,57,66.0,drinks,Made in Russia,2
1,46,83.0,drinks,Made in Russia,2
2,45,58.0,drinks,Made in Russia,2
3,63,2.0,drinks,Made in Russia,2
4,25,51.0,drinks,Made in Russia,2
5,75,77.0,drinks,Made in Russia,2
6,44,79.0,drinks,Made in Russia,2
7,91,23.0,drinks,Made in Russia,2


In [353]:
cur.execute('''
            drop procedure if exists get_orders;
            ''')

cur.execute('''
            CREATE procedure get_orders(
             IN in_time VARCHAR(50),
             IN in_drinks VARCHAR(50),
             IN in_description VARCHAR(50),
             IN in_date VARCHAR(50)
            )
            BEGIN
                SET @sql = CONCAT('SELECT
                      ps.product_id,
                      SUM(ps.quantity),
                      pi.properties,
                      pi.description,
                      ', in_time, '(date_sales) AS time
                      FROM products_sales ps
                      LEFT JOIN products_info pi ON pi.product_id = ps.product_id
                      WHERE 
                      pi.properties = "', in_drinks, '"
                      AND pi.description = "', in_description, '"
                      AND DATE_FORMAT(ps.date_sales, ''%Y-%m'') = "', in_date, '"
                      GROUP BY ps.product_id, time, pi.properties, pi.description');
                      
                PREPARE stmt FROM @sql;
                EXECUTE stmt;
                DEALLOCATE PREPARE stmt;
            END;
            ''')

cur.callproc('get_orders', ['MONTH', 'drinks', 'Made in Russia', '2023-02'])
result = []
for result_set in cur.stored_results():
    result.append(result_set.fetchall())
df = pd.DataFrame(result[0], columns=['product_id', 'sum', 'properties', 'description', 'time'])
df

,product_id,sum,properties,description,time
0,57,66,drinks,Made in Russia,2
1,46,83,drinks,Made in Russia,2
2,45,58,drinks,Made in Russia,2
3,63,2,drinks,Made in Russia,2
4,25,51,drinks,Made in Russia,2
5,75,77,drinks,Made in Russia,2
6,44,79,drinks,Made in Russia,2
7,91,23,drinks,Made in Russia,2


#### Даем права пользователю 'manager' на запуск процедуры 'get_orders'

In [ ]:
cur.execute('''
            GRANT EXECUTE ON PROCEDURE otus_db.get_orders TO 'manager'@'localhost';
            ''')